This notebook is dedicated to getting task vectors.

In [ ]:
import torch
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms
from transformers import BertTokenizer, BertModel
import math
import torch.nn.functional as F

!pip install tqdm
from tqdm import tqdm
import time



from google.colab import drive
drive.mount('/content/drive')

%cd /content/drive/MyDrive/Sapienza/Y1/DL & AAI/Project

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/.shortcut-targets-by-id/1fz0aq2n-SyUbVYiAvJLNUhWeDGqsiVmb/Sapienza/Y1/DL & AAI/Project


# Model Architectures

In [ ]:
class ResnetClassifier(nn.Module):
  def __init__(self, out_size=100, initialize="xavier"):
    super(ResnetClassifier, self).__init__()
    self.pretrained_model = models.resnet18(pretrained=True)
    self.LN = nn.LayerNorm(1000)
    self.fc = nn.Linear(1000, out_size)

    if initialize == "kaiming":
      init.kaiming_uniform_(self.fc.weight, mode='fan_out')
      init.constant_(self.fc.bias, 0)
    if initialize == "xavier":
      init.xavier_uniform_(self.fc.weight)
      init.constant_(self.fc.bias, 0)


  def forward(self, x):
      x = self.pretrained_model(x)
      x = self.LN(x)
      x = self.fc(x)
      return x

  def forward_backbone(self, x):
      x = self.pretrained_model(x)
      return x

In [ ]:
class ResnetFingerphotoRecognizer(nn.Module):
  def __init__(self, num_classes=100, hidden_size=256, initialize="xavier"):
    super(ResnetFingerphotoRecognizer, self).__init__()
    self.conv_1to3 = nn.Conv2d(1, 3, kernel_size=1, stride=1, padding=0)
    self.pretrained_model = models.resnet18(pretrained=True)
    #self.intermediate_norm = nn.BatchNorm1d(1000, affine=False) # batch norm doesn't work when batch size = 1, so let's use layerNorm
    self.intermediate_norm = nn.LayerNorm(1000)
    self.projector = nn.Linear(1000, hidden_size)
    self.final_norm = nn.LayerNorm(hidden_size)
    self.dropout = nn.Dropout(0.2)
    self.activation = nn.GELU()
    self.fc = nn.Linear(hidden_size, num_classes)
    self.softmax = nn.Softmax(dim=1)

    if initialize == "kaiming":
      init.kaiming_uniform_(self.projector.weight, mode='fan_in', nonlinearity='leaky_relu')
      init.kaiming_uniform_(self.fc.weight, mode='fan_out')
      init.constant_(self.fc.bias, 0)
    if initialize == "xavier":
      init.xavier_uniform_(self.projector.weight)
      init.xavier_uniform_(self.fc.weight)
      init.constant_(self.fc.bias, 0)


  def forward(self, x):
      if x.size()[1] == 1:
        x = self.conv_1to3(x)
      x = self.pretrained_model(x)
      x = self.intermediate_norm(x)
      x = self.projector(x)
      x = self.final_norm(x)
      x = self.activation(x)
      x = self.dropout(x)
      x = self.fc(x)
      x = self.softmax(x)
      return x
  def forward_logits(self, x):
      if x.size()[1] == 1:
        x = self.conv_1to3(x)
      x = self.pretrained_model(x)
      x = self.intermediate_norm(x)
      #x = self.activation(x)
      x = self.projector(x)
      x = self.final_norm(x)
      x = self.activation(x)
      x = self.dropout(x)
      x = self.fc(x)
      return x
  def forward_projector(self, x):
      if x.size()[1] == 1:
        x = self.conv_1to3(x)
      x = self.pretrained_model(x)
      x = self.intermediate_norm(x)
      x = self.projector(x)
      return x
  def forward_backbone(self, x):
      if x.size()[1] == 1:
        x = self.conv_1to3(x)
      x = self.pretrained_model(x)
      return x

# Helper Functions

In [ ]:
# It takes two architecturally identical models and returns pretrained - finetuned in a dictionary
def get_task_vector(pretrained, finetuned):
  assert type(pretrained) == type(finetuned)

  TV = {} # a dictionary where KEYS = weight matrices and VALUES = (finetuned weights - pretrained weights)
  for (name, pretrained_param), (_, fine_tuned_param) in zip(pretrained.named_parameters(), finetuned.named_parameters()):
      TV[name] = fine_tuned_param.data - pretrained_param.data

  return TV


def get_task_buffer(pretrained, finetuned):
  assert type(pretrained) == type(finetuned)

  TB = {} # a dictionary where KEYS = weight matrices and VALUES = (finetuned weights - pretrained weights)
  for (name, pretrained_buf), (_, fine_tuned_buf) in zip(pretrained.named_buffers(), finetuned.named_buffers()):
      TB[name] = fine_tuned_buf.data - pretrained_buf.data

  return TB


def get_weight_matrix(model, layer_name):
    """
    Get the weight matrix of a specific layer by name.

    Args:
    - model: The PyTorch model
    - layer_name: Name of the layer whose weight matrix is needed

    Returns:
    - The weight matrix tensor or None if not found
    """
    for name, param in model.named_parameters():
        if name == layer_name:
            return param
    return "No Layer Name Found!"

def are_tensors_equal(tensor1, tensor2, tol=1e-6):
    """
    Check if two tensors are equal within some tolerance.

    Parameters:
    tensor1 (torch.Tensor): First tensor to compare.
    tensor2 (torch.Tensor): Second tensor to compare.
    tol (float): Tolerance for the comparison. Default is 1e-6.

    Returns:
    bool: True if tensors are equal within the given tolerance, False otherwise.
    """
    if tensor1.shape != tensor2.shape:
        return False
    return torch.allclose(tensor1, tensor2, atol=tol, rtol=0)


def are_models_equal(model1, model2):
  for (name, par), (name2, par2) in zip(model1.named_parameters(), model2.named_parameters()):
    if(name != name2):
      print("Names not equal!")
      print(name, "   ",name2)
      return False
    if not are_tensors_equal(par, par2):
      print("Weights not equal!")
      print(name, "   ",name2)
      return False
  return True

# CIFAR-10
We will fine-tune our model on CIFAR-10.

In [ ]:
# Load CIFAR-10 dataset
transform = transforms.Compose([
    transforms.Resize(224),  # Resize images to 224x224 as expected by ResNet-18
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=256, shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=256, shuffle=False, num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [ ]:
model = ResnetClassifier(out_size=10, initialize="xavier").to("cuda")
model.to(device)

# Define loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs = 8

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 99.9MB/s]


In [ ]:
for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    start_epoch_time = time.time()  # Start time for the epoch

    # Wrap the DataLoader with tqdm
    pbar = tqdm(enumerate(trainloader), total=len(trainloader), desc=f'Epoch [{epoch+1}/{num_epochs}]')

    for batch_idx, (inputs, labels) in pbar:
        batch_start_time = time.time()  # Start time for the batch

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        batch_end_time = time.time()  # End time for the batch
        batch_duration = batch_end_time - batch_start_time

        # Update tqdm description with batch timing
        pbar.set_postfix(Batch_Time=f'{batch_duration:.2f}s', Loss=f'{loss.item():.4f}')

    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_duration = time.time() - start_epoch_time  # End time for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}] completed in {epoch_duration:.2f} seconds, Average Loss: {epoch_loss:.4f}')

    # Validation step (optional)
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy after Epoch [{epoch+1}/{num_epochs}]: {100 * correct / total:.2f}%')

print('Finished Training')

/usr/lib/python3.10/multiprocessing/popen_fork.py:66: RuntimeWarning: os.fork() was called. os.fork() is incompatible with multithreaded code, and JAX is multithreaded, so this will likely lead to a deadlock.
  self.pid = os.fork()
Epoch [1/8]: 100%|██████████| 97/97 [02:56<00:00,  1.82s/it, Batch_Time=1.37s, Loss=0.2805]

Epoch [1/8] completed in 176.69 seconds, Average Loss: 0.4641


Validation Accuracy after Epoch [1/8]: 87.18%


Epoch [2/8]: 100%|██████████| 97/97 [02:54<00:00,  1.79s/it, Batch_Time=1.39s, Loss=0.1567]

Epoch [2/8] completed in 174.08 seconds, Average Loss: 0.1846


...

Epoch [7/8] completed in 174.35 seconds, Average Loss: 0.0480


Validation Accuracy after Epoch [7/8]: 89.33%


Epoch [8/8]: 100%|██████████| 97/97 [02:54<00:00,  1.80s/it, Batch_Time=1.36s, Loss=0.0724]

Epoch [8/8] completed in 174.40 seconds, Average Loss: 0.0427


Validation Accuracy after Epoch [8/8]: 89.07%
Finished Training


In [ ]:
# Save the task vector of CIFAR-10 model
pretrained_model = models.resnet18(pretrained=True).to(device)

CIFAR10_TV = get_task_vector(pretrained_model, model.pretrained_model)
CIFAR10_TB = get_task_buffer(pretrained_model, model.pretrained_model)

torch.save(CIFAR10_TV, 'task vectors/CIFAR10_TV.pth')
torch.save(CIFAR10_TB, 'task vectors/CIFAR10_TB.pth')

CIFAR10_TV = torch.load('task vectors/CIFAR10_TV.pth')
torch.save(model, 'models/CIFAR10_model.pth')

# CIFAR-10 Dog Only
The goal is to extract the task vector for recognizing dogs only, so we can then make the model unlearn this very ability.

In [ ]:
# Define a transformation to preprocess the data
transform = transforms.Compose([
    transforms.Resize(224),  # Resize images to 224x224 as expected by ResNet-18
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])


trainset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=transform)
trainset.targets = [1 if label == 5 else 0 for label in trainset.targets]
trainloader = torch.utils.data.DataLoader(trainset, batch_size=512, shuffle=True, num_workers=4)


testset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=transform)
testset.targets = [1 if label == 5 else 0 for label in testset.targets]
testloader = torch.utils.data.DataLoader(testset, batch_size=512, shuffle=False, num_workers=4)

Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:558: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Files already downloaded and verified


In [ ]:
CIFAR_dog_model = ResnetClassifier(out_size=2, initialize="xavier").to("cuda")
CIFAR_dog_model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(CIFAR_dog_model.parameters(), lr=0.001)
num_epochs = 5

In [ ]:
for epoch in range(num_epochs):
    CIFAR_dog_model.train()
    running_loss = 0.0
    start_epoch_time = time.time()  # Start time for the epoch

    # Wrap the DataLoader with tqdm
    pbar = tqdm(enumerate(trainloader), total=len(trainloader), desc=f'Epoch [{epoch+1}/{num_epochs}]')

    for batch_idx, (inputs, labels) in pbar:
        batch_start_time = time.time()  # Start time for the batch

        inputs, labels = inputs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = CIFAR_dog_model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)

        batch_end_time = time.time()  # End time for the batch
        batch_duration = batch_end_time - batch_start_time

        # Update tqdm description with batch timing
        pbar.set_postfix(Batch_Time=f'{batch_duration:.2f}s', Loss=f'{loss.item():.4f}')

    epoch_loss = running_loss / len(trainloader.dataset)
    epoch_duration = time.time() - start_epoch_time  # End time for the epoch
    print(f'Epoch [{epoch+1}/{num_epochs}] completed in {epoch_duration:.2f} seconds, Average Loss: {epoch_loss:.4f}')

    # Validation step (optional)
    CIFAR_dog_model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for inputs, labels in testloader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = CIFAR_dog_model(inputs)
            _, predicted = torch.max(outputs, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print(f'Validation Accuracy after Epoch [{epoch+1}/{num_epochs}]: {100 * correct / total:.2f}%')

print('Finished Training')

Epoch [1/5]: 100%|██████████| 98/98 [02:59<00:00,  1.83s/it, Batch_Time=1.00s, Loss=0.2147]

Epoch [1/5] completed in 180.05 seconds, Average Loss: 0.1864


Validation Accuracy after Epoch [1/5]: 95.37%


Epoch [2/5]: 100%|██████████| 98/98 [02:58<00:00,  1.82s/it, Batch_Time=0.98s, Loss=0.0938]

Epoch [2/5] completed in 178.95 seconds, Average Loss: 0.0933


...

Epoch [4/5] completed in 175.89 seconds, Average Loss: 0.0505


Validation Accuracy after Epoch [4/5]: 94.13%


Epoch [5/5]: 100%|██████████| 98/98 [02:53<00:00,  1.77s/it, Batch_Time=0.98s, Loss=0.0458]

Epoch [5/5] completed in 173.99 seconds, Average Loss: 0.0389


Validation Accuracy after Epoch [5/5]: 96.24%
Finished Training


In [ ]:
# Get the task vector of this model recognizing dogs

CIFAR10_dog_TV = get_task_vector(pretrained_model, CIFAR_dog_model.pretrained_model)
CIFAR10_dog_TB = get_task_buffer(pretrained_model, CIFAR_dog_model.pretrained_model)

torch.save(CIFAR10_dog_TV, 'task vectors/CIFAR_dog_TV.pth')
torch.save(CIFAR10_dog_TB, 'task vectors/CIFAR_dog_TB.pth')

CIFAR10_dog_TV = torch.load('task vectors/CIFAR_dog_TV.pth')

torch.save(CIFAR_dog_model, 'models/CIFAR_dog_model.pth')

# Fingerphoto Recognizer

In [ ]:
fingerphoto_model = ResnetFingerphotoRecognizer(num_classes=128, hidden_size=128, initialize=True).to(device)

model_path = "models/resnetSimCLR_128hidden.pth"
fingerphoto_model = torch.load(model_path)

"""x = torch.rand((8, 1, 123, 331))
fingerphoto_model.forward_logits(x).shape"""

'x = torch.rand((8, 1, 123, 331))\nfingerphoto_model.forward_logits(x).shape'

In [ ]:
# Get the task vector of fingerphoto recognizer

fingerphoto_TV = get_task_vector(pretrained_model, fingerphoto_model.pretrained_model)
fingerphoto_TB = get_task_buffer(pretrained_model, fingerphoto_model.pretrained_model)

torch.save(fingerphoto_TV, 'task vectors/fingerphoto_TV.pth')
torch.save(fingerphoto_TB, 'task vectors/fingerphoto_TB.pth')

fingerphoto_TV = torch.load('task vectors/fingerphoto_TV.pth')

# English QA

## Pretrained Model


In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("AnonymousSub/bert-base-uncased_squad2.0")
eng_QA_model = AutoModelForQuestionAnswering.from_pretrained("AnonymousSub/bert-base-uncased_squad2.0")

In [ ]:
pretrained_bert = BertModel.from_pretrained('bert-base-uncased')
type(pretrained_bert) == type(eng_QA_model.bert)

eng_QA_TV = get_task_vector(pretrained=pretrained_bert, finetuned=eng_QA_model.bert)
eng_QA_TB = get_task_buffer(pretrained=pretrained_bert, finetuned=eng_QA_model.bert)
get_weight_matrix(eng_QA_model.bert, "encoder.layer.11.output.dense.weight")[0][:5], get_weight_matrix(pretrained_bert, "encoder.layer.11.output.dense.weight")[0][:5]

(tensor([-0.0205,  0.0542, -0.0093, -0.0190,  0.0146], grad_fn=<SliceBackward0>),
 tensor([-0.0224,  0.0540, -0.0089, -0.0192,  0.0155], grad_fn=<SliceBackward0>))

In [ ]:
torch.save(eng_QA_TV, "task vectors/eng_QA_TV.pth")
torch.save(eng_QA_TB, "task vectors/eng_QA_TB.pth")

## Non-Pretrained Model (Alternative)
Here we train ourselves the model.

In [ ]:
!pip install transformers datasets torch
!pip install transformers[torch] accelerate -U
!pip install accelerate -U

from transformers import BertTokenizerFast, BertForQuestionAnswering, Trainer, TrainingArguments
from datasets import load_dataset

In [ ]:
eng_QA_model = BertForQuestionAnswering.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
Some weights of BertForQuestionAnswering were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['qa_outputs.bias', 'qa_outputs.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


### SQuAD Dataset

In [ ]:
# Load the SQuAD dataset
squad_dataset = load_dataset('squad')

# Initialize the fast tokenizer
tokenizer = BertTokenizerFast.from_pretrained('bert-base-uncased')

# Tokenize the dataset
def preprocess_function(examples):
    questions = [q.strip() for q in examples["question"]]
    inputs = tokenizer(
        questions,
        examples["context"],
        max_length=384,
        truncation="only_second",  # Truncate the context if it's too long
        return_offsets_mapping=True,  # Return the character-to-token mapping
        padding="max_length"
    )

    offset_mapping = inputs.pop("offset_mapping")
    answers = examples["answers"]

    start_positions = []
    end_positions = []

    for i, offsets in enumerate(offset_mapping):
        answer = answers[i]
        start_char = answer["answer_start"][0]
        end_char = start_char + len(answer["text"][0])
        sequence_ids = inputs.sequence_ids(i)

        # Find the start and end of the context
        context_start = sequence_ids.index(1)
        context_end = len(sequence_ids) - 1 - sequence_ids[::-1].index(1)

        # If the answer is not fully inside the context, label it as impossible
        if not (offsets[context_start][0] <= start_char and offsets[context_end][1] >= end_char):
            start_positions.append(0)
            end_positions.append(0)
        else:
            # Otherwise, find the start and end token indices of the answer
            start_positions.append(next(idx for idx, span in enumerate(offsets) if span[0] <= start_char < span[1]))
            end_positions.append(next(idx for idx, span in enumerate(offsets) if span[0] < end_char <= span[1]))

    inputs["start_positions"] = start_positions
    inputs["end_positions"] = end_positions
    return inputs

# Preprocess the training and validation datasets
tokenized_squad = squad_dataset.map(preprocess_function, batched=True, remove_columns=squad_dataset["train"].column_names)

Generating train split:   0%|          | 0/87599 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/10570 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Map:   0%|          | 0/87599 [00:00<?, ? examples/s]

Map:   0%|          | 0/10570 [00:00<?, ? examples/s]

### Training Setup

In [ ]:
training_args = TrainingArguments(
    output_dir='./results',          # output directory
    eval_strategy="epoch",     # evaluate each `epoch`
    learning_rate=2e-5,              # learning rate
    per_device_train_batch_size=32,  # batch size for training
    per_device_eval_batch_size=32,   # batch size for evaluation
    num_train_epochs=3,              # total number of training epochs
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=10,
)


trainer = Trainer(
    model=eng_QA_model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=tokenized_squad["train"],  # training dataset
    eval_dataset=tokenized_squad["validation"]  # evaluation dataset
)


In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


KeyboardInterrupt: 

In [ ]:
trainer.evaluate()

### Task Vector Retrieval

In [ ]:
pretrained_bert = BertModel.from_pretrained('bert-base-uncased')

In [ ]:
type(pretrained_bert) == type(eng_QA_model.bert)

eng_QA_TV = get_task_vector(pretrained=pretrained_bert, finetuned=eng_QA_model.bert)
eng_QA_TB = get_task_buffer(pretrained=pretrained_bert, finetuned=eng_QA_model.bert)


In [ ]:
get_weight_matrix(eng_QA_model.bert, "encoder.layer.11.output.dense.weight")[0][:5], get_weight_matrix(pretrained_bert, "encoder.layer.11.output.dense.weight")[0][:5]


(tensor([-0.0224,  0.0540, -0.0089, -0.0192,  0.0155], grad_fn=<SliceBackward0>),
 tensor([-0.0224,  0.0540, -0.0089, -0.0192,  0.0155], grad_fn=<SliceBackward0>))

In [ ]:
torch.save(eng_QA_TV, "task vectors/trained_eng_QA_TV.pth")
torch.save(eng_QA_TB, "task vectors/trained_eng_QA_TB.pth")

# English NLI

## Pretrained Model

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-MNLI")
eng_nli_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-MNLI")

## Test it

In [ ]:
# Example premises and hypotheses
premises = [
    "The cat is sleeping on the mat.",
    "John bought a new car yesterday.",
    "Sarah is reading a book in the library.",
    "The dog barked loudly in the backyard.",
    "The restaurant offers a variety of vegetarian dishes.",
    "The sun was shining brightly all day.",
    "Alice went to Paris for her vacation.",
    "He gets good grades."
]
hypotheses = [
    "The cat is on the mat.",
    "John has a new car.",
    "Sarah borrowed a book from the library.",
    "The dog is hungry.",
    "The restaurant is expensive.",
    "It was raining all day.",
    "Alice stayed at home during her vacation.",
    "He is does terribly at school."
]

# Tokenize the inputs
inputs = tokenizer(premises, hypotheses, return_tensors='pt', padding=True, truncation=True)

# Make predictions
with torch.no_grad():
    outputs = eng_nli_model(**inputs)

# The model outputs logits, which are raw, unnormalized scores for each class (entailment, neutral, contradiction)
logits = outputs.logits
print(logits)

# Get the predicted class
predicted_class = torch.argmax(logits, dim=1)

# Class labels are typically: 0 -> entailment, 1 -> neutral, 2 -> contradiction
labels = ['contradiction', 'entailment', 'neutral']
predictions = [labels[idx] for idx in predicted_class]
#predictions = [idx for idx in predicted_class]

for i, (premise, hypothesis) in enumerate(zip(premises, hypotheses)):
    print(f"Premise: {premise}")
    print(f"Hypothesis: {hypothesis}")
    print(f"Prediction: {predictions[i]}")
    print()


tensor([[ 0.8441,  1.4855, -2.1183],
        [-3.0855,  4.4583, -1.4998],
        [-0.0906, -2.9443,  2.7264],
        [-0.2227, -3.1297,  2.9272],
        [ 1.2584, -3.5468,  2.3568],
        [ 4.0614, -3.2293, -0.5090],
        [ 4.9704, -3.4101, -1.4471],
        [ 5.3476, -3.4714, -1.5295]])
Premise: The cat is sleeping on the mat.
Hypothesis: The cat is on the mat.
Prediction: entailment

Premise: John bought a new car yesterday.
Hypothesis: John has a new car.
Prediction: entailment

Premise: Sarah is reading a book in the library.
Hypothesis: Sarah borrowed a book from the library.
Prediction: neutral

Premise: The dog barked loudly in the backyard.
Hypothesis: The dog is hungry.
Prediction: neutral

Premise: The restaurant offers a variety of vegetarian dishes.
Hypothesis: The restaurant is expensive.
Prediction: neutral

Premise: The sun was shining brightly all day.
Hypothesis: It was raining all day.
Prediction: contradiction

Premise: Alice went to Paris for her vacation.
H

## Task Vector Retrieval

In [ ]:
pretrained_bert = BertModel.from_pretrained('bert-base-uncased')
type(pretrained_bert) == type(eng_nli_model.bert)

eng_nli_TV = get_task_vector(pretrained=pretrained_bert, finetuned=eng_nli_model.bert)
eng_nli_TB = get_task_buffer(pretrained=pretrained_bert, finetuned=eng_nli_model.bert)
get_weight_matrix(eng_nli_model.bert, "encoder.layer.11.output.dense.weight")[0][:5], get_weight_matrix(pretrained_bert, "encoder.layer.11.output.dense.weight")[0][:5]


(tensor([-0.0229,  0.0535, -0.0090, -0.0180,  0.0144], grad_fn=<SliceBackward0>),
 tensor([-0.0224,  0.0540, -0.0089, -0.0192,  0.0155], grad_fn=<SliceBackward0>))

In [ ]:
torch.save(eng_nli_TV, "task vectors/eng_nli_TV.pth")
torch.save(eng_nli_TB, "task vectors/eng_nli_TB.pth")

# Italian QA

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForQuestionAnswering

tokenizer = AutoTokenizer.from_pretrained("mrm8488/bert-italian-finedtuned-squadv1-it-alfa")
ita_QA_model = AutoModelForQuestionAnswering.from_pretrained("mrm8488/bert-italian-finedtuned-squadv1-it-alfa")

tokenizer_config.json:   0%|          | 0.00/40.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/442 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/235k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at mrm8488/bert-italian-finedtuned-squadv1-it-alfa were not used when initializing BertForQuestionAnswering: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight']
- This IS expected if you are initializing BertForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
pretrained_bert = BertModel.from_pretrained('bert-base-uncased')
type(pretrained_bert) == type(ita_QA_model.bert)

# only compute task vector from the bert's encoder, since the embedder are different between English and Italian
ita_QA_TV = get_task_vector(pretrained=pretrained_bert.encoder, finetuned=ita_QA_model.bert.encoder)
ita_QA_TB = get_task_buffer(pretrained=pretrained_bert.encoder, finetuned=ita_QA_model.bert.encoder)

get_weight_matrix(ita_QA_model.bert, "encoder.layer.11.output.dense.weight")[0][:5], get_weight_matrix(pretrained_bert, "encoder.layer.11.output.dense.weight")[0][:5]

torch.save(ita_QA_TV, "task vectors/ita_QA_TV.pth")
torch.save(ita_QA_TB, "task vectors/ita_QA_TB.pth")

In [ ]:
get_weight_matrix(ita_QA_model.bert, "encoder.layer.11.output.dense.weight")[0][:5], get_weight_matrix(pretrained_bert, "encoder.layer.11.output.dense.weight")[0][:5]


(tensor([-0.0110,  0.0430, -0.0137, -0.0022, -0.0352], grad_fn=<SliceBackward0>),
 tensor([-0.0224,  0.0540, -0.0089, -0.0192,  0.0155], grad_fn=<SliceBackward0>))

# English Sentiment Analysis

In [ ]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("Camelia7v/bert-sentiment-analysis-model-25k-samples")
eng_SA_model = AutoModelForSequenceClassification.from_pretrained("Camelia7v/bert-sentiment-analysis-model-25k-samples")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/348 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/711k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/727 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

In [ ]:
pretrained_bert = BertModel.from_pretrained('bert-base-uncased')
type(pretrained_bert) == type(eng_SA_model.bert)

# only compute task vector from the bert's encoder, since the embedder are different between English and Italian
eng_SA_TV = get_task_vector(pretrained=pretrained_bert.encoder, finetuned=eng_SA_model.bert.encoder)
eng_SA_TB = get_task_buffer(pretrained=pretrained_bert.encoder, finetuned=eng_SA_model.bert.encoder)

get_weight_matrix(eng_SA_model.bert, "encoder.layer.11.output.dense.weight")[0][:5], get_weight_matrix(pretrained_bert, "encoder.layer.11.output.dense.weight")[0][:5],eng_SA_TV["layer.11.output.dense.weight"][0][:5]

(tensor([-0.0215,  0.0533, -0.0095, -0.0188,  0.0148], grad_fn=<SliceBackward0>),
 tensor([-0.0224,  0.0540, -0.0089, -0.0192,  0.0155], grad_fn=<SliceBackward0>),
 tensor([ 0.0008, -0.0008, -0.0005,  0.0004, -0.0007]))

In [ ]:
torch.save(eng_SA_TV, "task vectors/eng_SA_TV.pth")
torch.save(eng_SA_TB, "task vectors/eng_SA_TB.pth")